In [ ]:
# DEVELOP A PYTHON APPLICATION TO COLLECT , STORE, AND ANALYZE CUSTOMER FEEDBACK.THE PROJECT SHOULD PERFORM BASIC SENTIMENT ANALYSIS ON CUSTOMER
# REVIEWS AND PROVIDE INSIGHTS USING PANDAS AND NUMPY.

In [ ]:
# FUNCTIONALITIES:

# 1.OBJECT-ORIENTED DESIGN:

# DESIGN CLASSES FOR CUSTOMER, REVIEW, SENTIMENT ANALYZER.
# EACH CUSTOMER CAN LEAVE MULTIPLE REVIEWS FOR PRODUCTS.
# USE OOP PRINCIPLES LIKE INHERITANCE AND POLYMORPHISM.

# 2.FILE HANDLING:

# STORE CUSTOMER INFORMATION AND REVIEWS IN A FILE (reviews.csv).
# EACH ENTRY SHOULD CONTAIN DETAILS LIKE CUSTOMER ID,PRODUCT ID,REVIEW,DATE, AND RATING.
# USE CSV OR JSON FILES  FOR STORAGE AND RETRIEVAL.

# 3.SENTIMENT ANALYSIS USING DECORATORS:

# IMPLEMENT A SIMPLE SENTIMENT ANALYSIS USING PYTHON'S TEXTBLOB OR NLTK TO
# CLASSIFY REVIEWS AS POSITIVE, NEGATIVE, OR NEUTRAL.
# USE A DECORATOR TO PREPROCESS THE REVIEWS (like lowercasing,removing punctuation) BEFORE PASSING THEM TO THE SENTIMENT ANALYZER.


In [19]:
import csv
import string
from datetime import datetime
from textblob import TextBlob

# Customer class to hold customer information and manage reviews
class Customer:
    def __init__(self, customer_id, name, email):
        self.customer_id = customer_id
        self.name = name
        self.email = email
        self.reviews = []  # Store all reviews made by the customer

    def leave_review(self, product, rating, comment):
        review = Review(self.customer_id, product.product_id, rating, comment)
        review.save_to_file()
        product.add_review(review)
        self.reviews.append(review)
        return review

# Review class to store individual review details
class Review:
    def __init__(self, customer_id, product_id, rating, comment, date=None):
        self.customer_id = customer_id
        self.product_id = product_id
        self.rating = rating
        self.comment = comment
        self.date = date if date else datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def save_to_file(self, filename='reviews.csv'):
        """Save the review to a CSV file."""
        with open(filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([self.customer_id, self.product_id, self.comment, self.date, self.rating])

    @staticmethod
    def load_from_file(filename='reviews.csv'):
        """Load reviews from a CSV file."""
        reviews = []
        with open(filename, mode='r') as file:
            reader = csv.reader(file)
            for row in reader:
                customer_id, product_id, comment, date, rating = row
                reviews.append(Review(customer_id, product_id, rating, comment, date))
        return reviews

# Product class to store product information and manage reviews for products
class Product:
    def __init__(self, product_id, name, price):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.reviews = []  # Store product reviews

    def add_review(self, review):
        self.reviews.append(review)

# SentimentAnalyzer base class
class SentimentAnalyzer:
    def analyze_sentiment(self, review):
        """Analyze the sentiment of a review - to be implemented by subclasses."""
        raise NotImplementedError("Subclasses should implement this method.")

# Sentiment analysis decorator for preprocessing reviews
def preprocess_review(func):
    def wrapper(review):
        review.comment = review.comment.lower().translate(str.maketrans('', '', string.punctuation))
        return func(review)
    return wrapper

# Simple sentiment analyzer using TextBlob (inheritance & polymorphism)
class TextBlobSentimentAnalyzer(SentimentAnalyzer):
    @preprocess_review
    def analyze_sentiment(self, review):
        blob = TextBlob(review.comment)
        polarity = blob.sentiment.polarity
        if polarity > 0:
            return "Positive"
        elif polarity < 0:
            return "Negative"
        else:
            return "Neutral"


In [20]:
from textblob import TextBlob

# Reviews for testing sentiment analysis
reviews = [
    "This product is amazing! I absolutely love it!",  # Positive review
    "The product is okay, nothing special.",           # Neutral review
    "I hate this product. It was a waste of money."    # Negative review
]

# Function to classify sentiment
def classify_review(review_text):
    blob = TextBlob(review_text)
    polarity = blob.sentiment.polarity

    # Classify sentiment based on polarity score
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"

# Test with sample reviews
for review in reviews:
    sentiment = classify_review(review)
    print(f"Review: \"{review}\" --> Sentiment: {sentiment}")


Review: "This product is amazing! I absolutely love it!" --> Sentiment: Positive
Review: "The product is okay, nothing special." --> Sentiment: Positive
Review: "I hate this product. It was a waste of money." --> Sentiment: Negative


In [ ]:
# PANDAS-BASED DATA ANALYSIS:

# LOAD THE FEEDBACK DATA USING PANDAS.
# GENERATE THE FOLLOWING INSIGHTS:
# 1.OVERALL CUSTOMER SATISFACTION(BASED ON AVERAGE SENTIMENT SCORE).
# 2. PRODUCT RATINGS BASED ON CUSTOMER FEEDBACK.
# 3. FIND PRODUCTS WITH THE MOST POSITIVE AND NEGATIVE REVIEWS.

In [21]:
# Customer ID,Product ID,Review,Date,Rating
1,101,"Great product, I love it!","2024-09-11",5
2,102,"The product was okay.","2024-09-11",3
3,101,"Terrible product. Waste of money!","2024-09-11",1
4,103,"This is the best purchase I have made.","2024-09-11",5
5,101,"Not bad, but could be better.","2024-09-11",3


(5, 101, 'Not bad, but could be better.', '2024-09-11', 3)

In [22]:
import pandas as pd
from textblob import TextBlob

# Load the review data from a CSV file
def load_reviews_to_dataframe(filename='reviews.csv'):
    """Load reviews from CSV into a pandas DataFrame."""
    df = pd.read_csv(filename, names=['Customer ID', 'Product ID', 'Review', 'Date', 'Rating'])
    return df

# Function to classify the sentiment of a review using TextBlob
def classify_sentiment(review):
    blob = TextBlob(review)
    polarity = blob.sentiment.polarity  # Polarity ranges from -1 (negative) to 1 (positive)
    return polarity

# Generate insights from the review data
def generate_insights(df):
    # 1. Overall Customer Satisfaction (average sentiment score)
    df['Sentiment Score'] = df['Review'].apply(classify_sentiment)
    overall_satisfaction = df['Sentiment Score'].mean()

    # 2. Product Ratings Based on Customer Feedback (average rating per product)
    product_ratings = df.groupby('Product ID')['Rating'].mean()

    # 3. Find Products with the Most Positive and Negative Reviews
    most_positive_product = df.groupby('Product ID')['Sentiment Score'].mean().idxmax()
    most_negative_product = df.groupby('Product ID')['Sentiment Score'].mean().idxmin()

    # Return results as a dictionary
    return {
        'Overall Customer Satisfaction': overall_satisfaction,
        'Product Ratings': product_ratings,
        'Most Positive Product': most_positive_product,
        'Most Negative Product': most_negative_product
    }

# Load the data and generate insights
df = load_reviews_to_dataframe('reviews.csv')
insights = generate_insights(df)

# Print the generated insights
print("Overall Customer Satisfaction (Average Sentiment Score):", insights['Overall Customer Satisfaction'])
print("Product Ratings (Average Rating per Product):")
print(insights['Product Ratings'])
print("Product with the Most Positive Reviews:", insights['Most Positive Product'])
print("Product with the Most Negative Reviews:", insights['Most Negative Product'])


Overall Customer Satisfaction (Average Sentiment Score): 1.0
Product Ratings (Average Rating per Product):
Product ID
P001    5.0
Name: Rating, dtype: float64
Product with the Most Positive Reviews: P001
Product with the Most Negative Reviews: P001


In [ ]:
# NUMPY-BASED CALCULATIONS:

# USE NUMPY TO CALCULATE AVERAGE RATINGS FOER EACH PRODUCT.
# USE NUMPY TO PERFORM STATISTICAL OPERATIONS(MEAN, MEDIAN, MODE) ON RATINGS AND SENTIMENT SCORES.

In [23]:
import numpy as np

def numpy_based_stats(df):
    ratings = df['Rating'].values
    mean_rating = np.mean(ratings)
    median_rating = np.median(ratings)
    mode_rating = pd.Series(ratings).mode().values[0]  # Using pandas mode for easier handling
    return mean_rating, median_rating, mode_rating


In [ ]:
# TESTING AND ERROR HANDLING:

# WRITE UNIT TESTS FOR MAJOR COMPONENTS LIKE ADDING REVIEWS,SENTIMENT ANALYSIS, AND FEEDBACK STORAGE.
# INCLUDE PROPER ERROR HANDLING(e.g.,handling empty reviews,invalid data formats).

In [24]:
import unittest

class TestReviewSystem(unittest.TestCase):
    def test_review_creation(self):
        review = Review(1, 101, 5, "Great product!")
        self.assertEqual(review.rating, 5)

    def test_empty_review(self):
        with self.assertRaises(ValueError):
            Review(1, 101, 5, "")

    def test_sentiment_analysis(self):
        review = Review(1, 101, 5, "Great product!")
        analyzer = TextBlobSentimentAnalyzer()
        sentiment = analyzer.analyze_sentiment(review)
        self.assertEqual(sentiment, "Positive")

if __name__ == '__main__':
    unittest.main()


E
ERROR: /root/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/root/'

----------------------------------------------------------------------
Ran 1 test in 0.004s

FAILED (errors=1)


SystemExit: True

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
